In [1]:
!pip install folium

In [2]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
import folium
import time

from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
from selenium import webdriver

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [12]:
#웹드라이버 설정
chrome_options = webdriver.ChromeOptions()
#chrome_oprtions.add_argument('headless')#창이 떠서 작업하는것이 아닌 백그라운드에서 작업

driver = webdriver.Chrome("../driver/chromedriver.exe", options=chrome_options)
driver.implicitly_wait(10)


#스타벅스 지도 페이지 오픈
starbucks_map_url = "https://www.starbucks.co.kr/store/store_map.do"
driver.get(starbucks_map_url)

C:\Users\pjh23\AppData\Local\Temp\ipykernel_57636\2413896392.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("../driver/chromedriver.exe", options=chrome_options)


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 105
Current browser version is 107.0.5304.87 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
Backtrace:
	Ordinal0 [0x00B2DF13+2219795]
	Ordinal0 [0x00AC2841+1779777]
	Ordinal0 [0x009D423D+803389]
	Ordinal0 [0x009F64AC+943276]
	Ordinal0 [0x009F19F0+924144]
	Ordinal0 [0x009EF179+913785]
	Ordinal0 [0x00A236B9+1128121]
	Ordinal0 [0x00A2331A+1127194]
	Ordinal0 [0x00A1E616+1107478]
	Ordinal0 [0x009F7F89+950153]
	Ordinal0 [0x009F8F56+954198]
	GetHandleVerifier [0x00E22CB2+3040210]
	GetHandleVerifier [0x00E12BB4+2974420]
	GetHandleVerifier [0x00BC6A0A+565546]
	GetHandleVerifier [0x00BC5680+560544]
	Ordinal0 [0x00AC9A5C+1808988]
	Ordinal0 [0x00ACE3A8+1827752]
	Ordinal0 [0x00ACE495+1827989]
	Ordinal0 [0x00AD80A4+1867940]
	BaseThreadInitThunk [0x762F6739+25]
	RtlGetFullPathName_UEx [0x772A8FD2+1218]
	RtlGetFullPathName_UEx [0x772A8F9D+1165]


In [ ]:
#윈도우 화면 최대화
driver.maximize_window()

In [ ]:
starbucks_shift_to_seoul_css = "#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,starbucks_shift_to_seoul_css))).click()

In [ ]:
#스타벅스 서울 클릭
starbucks_seoul_css = "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,starbucks_seoul_css))).click()

In [ ]:
#스타벅스 서울 전체 클릭
starbucks_seoul_all_css = "#mCSB_2_container > ul > li:nth-child(1) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,starbucks_seoul_all_css))).click()

In [ ]:
#스타벅스 서울 전체 HTML 피싱
html = driver.page_source
soup = BeautifulSoup(html,"html.parser")

In [ ]:
container = soup.find('div', id='mCSB_3_container')
li_all = container.find_all("li")
li_all[0]

In [ ]:
starbucks_data = []

for li in li_all:
    name = li.find('strong').text.strip()
    address= li.find('p').text.strip().replace("1522-3232","")
    gu = address.split(" ")[1]
    each = {
        "매장이름":name, "주소":address, "구":gu, '브랜드' : '스타벅스'
    }
    starbucks_data.append(each)

In [ ]:
len(starbucks_data)

In [ ]:
df_starbucks = pd.DataFrame(starbucks_data)
df_starbucks.tail()

In [ ]:
gu_list = list(df_starbucks['구'].unique())
len(gu_list)

In [ ]:
# 이디야 지도 페이지 오픈
ediya_map_url = "https://www.ediya.com/contents/find_store.html"
driver.get(ediya_map_url)

In [ ]:
#이디야 페이지 최대화
driver.maximize_window()

In [ ]:
# 이디야 주소 검색 클릭
ediya_address_css = "#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,ediya_address_css))).click() 

In [ ]:

#이디야 지도 서울 > 지역별 데이터 검색
for gu in tqdm_notebook(gu_list):
    
    ediya_search_keyword_css = "#keyword"
    
    #이디야 주소 검생어 초기화
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,ediya_search_keyword_css))).clear()
    
    # 검색어 입력
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,ediya_search_keyword_css))).send_keys(f"서울 {gu}")
    
    #이디야 주소 검색 버튼 클릭
    ediya_search_button_css = "#keyword_div > form > button"
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,ediya_search_button_css))).click()
    
    #이디야 구별 매장 정보 수집
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    ul_tag = soup.find('ul', id = "placesList")
    dl_all = ul_tag.find_all = ("dl")
    
    for dl in dl_all:
        name = dl.find("dt").text.strip()
        address = dl.find("dd").text.strip()
        gu = address.text.strip(' ')

In [ ]:
# 이디야 주소 검색 클릭
ediya_address_css = "#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,ediya_address_css))).click()

In [ ]:
# 검색어 입력
ediya_search_keyword_css = "#keyword"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,ediya_search_keyword_css))).send_keys("서울 강남구")

In [ ]:
#이디야 주소 검색 버튼 클릭
ediya_search_button_css = "#keyword_div > form > button"
WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR,ediya_search_button_css))).click()